## Creating the Optimal Gov's Ball Scedule

In [35]:
#Bring in the neccessary libraries 
import pandas as pd
from io import StringIO
import urllib
from bs4 import BeautifulSoup 
import sqlite3 
import time
import numpy 
import signal
from urllib.request import Request, urlopen

### Get Gov's Ball Artist Info

In [2]:
#Grab the artist info from the Gov's ball website
with urllib.request.urlopen("https://www.governorsballmusicfestival.com/lineup/interactive-lineup/") as url:
    s = url.read()
soup = BeautifulSoup(s, "lxml")


In [6]:
info = soup.findAll('div', {"class":"c-lineup__artist"})
#Put those names in a list, then a dataframe
govs_ball_data = pd.DataFrame([])
for i in info:
    artist = i.attrs['data-title']
    date = i.attrs["data-day-titles"]
    govs_ball_data = govs_ball_data.append(pd.DataFrame({'artist': artist, #Create the table
                                                         'date'  : date},
                                                  index=[0]),
                                     ignore_index=True)
govs_ball_data.head()

artist                    date
0               Tool    ["Sunday, June 4th"]
1  Chance The Rapper    ["Friday, June 2nd"]
2            Phoenix  ["Saturday, June 3rd"]
3   Childish Gambino  ["Saturday, June 3rd"]
4              Lorde    ["Friday, June 2nd"]

In [29]:
govs_ball_data["day"] = ((govs_ball_data["date"].str.slice(-5, -4)).astype(int) -1).astype(str)
govs_ball_data["date"] = govs_ball_data["date"].str.slice(2, -2)
govs_ball_data

artist                date day
0                          Tool    Sunday, June 4th   3
1             Chance The Rapper    Friday, June 2nd   1
2                       Phoenix  Saturday, June 3rd   2
3              Childish Gambino  Saturday, June 3rd   2
4                         Lorde    Friday, June 2nd   1
5                         Flume    Friday, June 2nd   1
6                  Wu-Tang Clan  Saturday, June 3rd   2
7                   Wiz Khalifa    Sunday, June 4th   3
8                         Logic    Sunday, June 4th   3
9             Cage The Elephant    Sunday, June 4th   3
10                   Marshmello  Saturday, June 3rd   2
11  Mark Ronson vs Kevin Parker  Saturday, June 3rd   2
12                  Beach House    Friday, June 2nd   1
13                  ScHoolboy Q    Friday, June 2nd   1
14                          AIR    Sunday, June 4th   3
15               The Avalanches    Sunday, June 4th   3
16                 Rae Sremmurd  Saturday, June 3rd   2
17       The Head and The Heart  Saturday, June 3rd   2
18                      Tove Lo    Friday, June 2nd   1
19                   Phantogram    Sunday, June 4th   3
20              Franz Ferdinand    Sunday, June 4th   3
21                        BANKS  Saturday, June 3rd   2
22                    A$AP Ferg  Saturday, June 3rd   2
23                Local Natives  Saturday, June 3rd   2
24                 Majid Jordan    Friday, June 2nd   1
25                  Mac DeMarco    Sunday, June 4th   3
26                 RÜFÜS DU SOL  Saturday, June 3rd   2
27                           YG  Saturday, June 3rd   2
28               Parquet Courts    Sunday, June 4th   3
29                   Charli XCX    Friday, June 2nd   1
..                          ...                 ...  ..
36                     Warpaint    Sunday, June 4th   3
37            Car Seat Headrest  Saturday, June 3rd   2
38                       Skepta    Sunday, June 4th   3
39                  Saint Motel  Saturday, June 3rd   2
40              The Strumbellas    Friday, June 2nd   1
41       Francis and the Lights    Friday, June 2nd   1
42                      Stormzy  Saturday, June 3rd   2
43                      GRYFFIN    Sunday, June 4th   3
44                    The Range  Saturday, June 3rd   2
45                     Dua Lipa  Saturday, June 3rd   2
46                         EDEN    Sunday, June 4th   3
47                    Zane Lowe    Sunday, June 4th   3
48                  The Orwells    Sunday, June 4th   3
49             Judah & the Lion    Friday, June 2nd   1
50                         MUNA    Friday, June 2nd   1
51                A R I Z O N A  Saturday, June 3rd   2
52                    SAINt JHN    Sunday, June 4th   3
53                Michael Blume    Friday, June 2nd   1
54                 Jessie Reyez  Saturday, June 3rd   2
55                      Lo Moon  Saturday, June 3rd   2
56                       Kaiydo    Sunday, June 4th   3
57                    Roosevelt    Friday, June 2nd   1
58               Barns Courtney    Sunday, June 4th   3
59                     Blossoms    Friday, June 2nd   1
60                    Ron Gallo    Sunday, June 4th   3
61                       Welles  Saturday, June 3rd   2
62                  Mondo Cozmo    Sunday, June 4th   3
63                  Tkay Maidza    Friday, June 2nd   1
64                       Jenaux    Friday, June 2nd   1
65                         VANT  Saturday, June 3rd   2

[66 rows x 3 columns]

#### Add Names to a SQLite Database

In [30]:
con = sqlite3.connect("pitchfork-data.db")

In [31]:
govs_ball_data.to_sql("govs_ball_data", con,if_exists='replace')

## Pitchfork Crawler

The crawling happens in two distinct stages. In Stage 1, the code loops throught the reviews page on pitchfork.com to find links to all the reviews. Stage 2 goes to each link and pull various bits of information. There's lots more to pull, but this is a solid starting place. 

In [62]:
links = [] #Create an empty list to fill with links

In [69]:
#Stage 1
AVERAGE_SECONDS_BETWEEN_REQUESTS = 5 #Don't go too hard on Pitchfork's servers

for i in range(1074,1500): #Use the range function to decide how many pages you want to go through 
    page_no = str(i)
    link = ('http://pitchfork.com/reviews/albums/?page=' + page_no) #create the link
    req = Request(link, headers={ 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }) #Mask the bot
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info = soup.findAll('a', {"class":"album-link"}) #pull the album link
    for j in info:
        links.append(j.attrs['href']) #grab all the link attributes
    time.sleep(numpy.random.exponential(AVERAGE_SECONDS_BETWEEN_REQUESTS, 1))  # pause between server requests
    print (i)
    


1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273


KeyboardInterrupt: 

In [70]:
len(links)

16716

In [71]:
link_table = pd.DataFrame([])
link_table["links"] = links
#link_table.to_sql("link_table", con)
link_table.head()

links
0                 /reviews/albums/22936-the-tourist/
1            /reviews/albums/22003-planetary-prince/
2            /reviews/albums/22939-machine-response/
3  /reviews/albums/22878-various-artists-outro-te...
4                      /reviews/albums/22978-hndrxx/

In [78]:
album_table = pd.DataFrame([]) #Create an empty dataframe that'll hold the info for each album

In [84]:
BASE_URL = 'http://www.pitchfork.com'
AVERAGE_SECONDS_BETWEEN_REQUESTS = 5

for i in links:
    link = BASE_URL + i
    req = Request(link, headers={ 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml") #same as above
    artist_info = soup.findAll('ul', {"class":"artist-links artist-list"}) #Artist Name
    album_info = soup.findAll('h1', {"class":"review-title"}) #Album Name
    score_info = soup.findAll('div', {"class":"score-circle"}) #Score
    pub_info = soup.findAll('span', {"class":"pub-date"}) # Publication Date
    genre_info = soup.findAll('ul', {"class":"genre-list before"}) #Genre
    for j in artist_info:
        artist = j.text
    for k in album_info:
        album = k.text
    for l in score_info:
        score = l.text
    for m in pub_info:
        pub_date = m.text
    for n in genre_info:
        genre = n.text
    print (artist, ", ", album)
    album_table = album_table.append(pd.DataFrame({'artist': artist, #Create the table
                                                   'album'  : album,
                                                   'score' : score,
                                                   'pub_date' : pub_date, 
                                                   'genre' : genre},
                                                  index=[0]),
                                     ignore_index=True)
    time.sleep(numpy.random.exponential(AVERAGE_SECONDS_BETWEEN_REQUESTS, 1))


album_table.head()

Clap Your Hands Say Yeah ,  The Tourist
Cameron Graves ,  Planetary Prince
Career Suicide ,  Machine Response
Various Artists ,  Outro Tempo: Electronic and Contemporary Music From Brazil, 1978-1992
Future ,  HNDRXX
PC Worship ,  Buried Wish
Aseethe ,  Hopes of Failure
Dams of the West ,  Youngish American
Vagabon ,  Infinite Worlds
Oddisee ,  The Iceberg
Sherwood & Pinch ,  Man Vs. Sofa
Karriem Riggins ,  Headnod Suite
Power Trip ,  Nightmare Logic
Los Campesinos! ,  Sick Scenes
Pissed Jeans ,  Why Love Now
Xiu Xiu ,  FORGET
Sun Kil Moon ,  Common as Light and Love Are Red Valleys of Blood
Stormzy ,  Gang Signs & Prayer
Entrance ,  Book of Changes
Six Organs of Admittance ,  Burning the Threshold
Flume ,  Skin Companion EP II
Thundercat ,  Drunk
The Feelies ,  In Between
Jonwayne ,  Rap Album Two
Six by Seven ,  The Closer You Get
Weezer ,  Weezer (Blue Album)
Kingdom ,  Tears in the Club
King Gizzard & The Lizard Wizard ,  Flying Microtonal Banana
Grails ,  Chalice Hymnal
Alison Krau

KeyboardInterrupt: 

In [85]:
#album_table.to_sql("album_table", con,if_exists='replace')
album_table

album  \
0                                          The Tourist   
1                                     Planetary Prince   
2                                     Machine Response   
3    Outro Tempo: Electronic and Contemporary Music...   
4                                               HNDRXX   
5                                          Buried Wish   
6                                     Hopes of Failure   
7                                    Youngish American   
8                                      Infinite Worlds   
9                                          The Tourist   
10                                    Planetary Prince   
11                                    Machine Response   
12                                         The Tourist   
13                                    Planetary Prince   
14                                    Machine Response   
15   Outro Tempo: Electronic and Contemporary Music...   
16                                              HNDRXX   
17                                         Buried Wish   
18                                    Hopes of Failure   
19                                   Youngish American   
20                                     Infinite Worlds   
21                                         The Iceberg   
22                                        Man Vs. Sofa   
23                                       Headnod Suite   
24                                     Nightmare Logic   
25                                         Sick Scenes   
26                                        Why Love Now   
27                                              FORGET   
28   Common as Light and Love Are Red Valleys of Blood   
29                                 Gang Signs & Prayer   
..                                                 ...   
75                                                Heba   
76                                            Prisoner   
77                Wildly Idle (Humble Before the Void)   
78   Being You Is Great, I Wish I Could Be You More...   
79                                       Undying Color   
80                                        DROGAS Light   
81   Studio One Rocksteady 2: The Soul of Young Jam...   
82                               Terrible Human Beings   
83                                         Drip Mental   
84                                   Celebrate Ornette   
85                       Lifestyles of the Laptop Café   
86                                    Memories Are Now   
87   Africa Airways Three: The Afro-Psych Excursion...   
88                                          Bad Timing   
89                                        Auto-Reverse   
90                                              Messes   
91                                                Blue   
92                          It's Good to Be Differ-Ent   
93                                           Snowdonia   
94                                       Bye Bye Berta   
95                                              Uptown   
96                                        Obsidian Arc   
97                                          I Decided.   
98                                     Little Fictions   
99                               The Drum Chord Theory   
100                                            Borders   
101                                                Fin   
102                                    After the Party   
103                             Themes for Dying Earth   
104                                   Life & Livin’ It   

                       artist                   genre          pub_date score  
0    Clap Your Hands Say Yeah                    Rock      March 4 2017   7.5  
1              Cameron Graves                    Jazz      March 4 2017   7.7  
2              Career Suicide                    Rock      March 4 2017   7.5  
3             Various Artists                    Rock      March 4 2017   8.8  
4                      Future                     Ra